<center>

# Implementation of New Message Encryption using Elliptic Curve Cryptography Over Finite Fields
</center>

Used article: [link](https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=9493519)

## Proposed method parameters

<center>

| Set-up Parameters | *n, a, b, p, G* |
|-------------------|-----------------|
| Key Generator     | *P_B, P_A, n_A, n_B, K* |
| Encryption | *E* = [*c_1c_2c_3...c_l*] |
| Decryption | *M* = [m_1m_2m_3...m_l] |
</center>

## 01. Set-up Parameters

A and B side should agree upon an elliptic curve *E_p(a,b)* over finite fields where *p* is a prime number, *a* and *b* are integers less than *p*.

Points are calculated according to the equation:
$$ y^2 = x^3 + ax + b \mod p$$

Which then needs to satisfy the condition:
$$ 4a^3+27b^2\ne 0 \mod p$$

In [2]:
# Example parameters (a,b,p) = (1, 37, 1286081)
import math

a = 1
b = 37
p = 1286081

# Checking if the equation satisfies the condition equality
condition_val = (4*a**3 + 27*b**2) % p

print(condition_val)

36967


In [57]:
# count points on the finite field E_1286081
def sqrt(val: int):
    return val**(1/2)

def count_y(x, a, b):
    aux = x**3 + a*x + b
    return aux

def point(x, y, a, b, p):
    result = (x**3 + a*x + b - y**2) % p
    if result == 0:
        # print("Point P({},{}) belongs to curve".format(x, y))
        return True
    else:
        # print("Result:", result)
        return False
    
def map_points(a, b, p):
    for y in reversed(range(p)):
        print("{:2d}".format(y), end=" ")
        for x in range(p):
            result = point(x, y, a, b, p)
            if result == True:
                print("+", end=" ")
            else:
                print(".", end=" ")
        print("")
        
    print("   ", end="")
    for x_label in range(p):
        if x_label >= 10:
            print("{:2d}".format(x_label), end="")
        else:
            print("{}".format(x_label), end=" ")

# print(point(5, 8, 0, 7, 17))

def list_points(a, b, p):
    points = []
    
    for x in range(p):
        for y in range(p):
            result = point(x, y, a, b, p)
            if result == True:
                points.append((x, y))
    return points

map_points(4, 9, 13)

12 . + . . . . . . . . . . . 
11 . . . . . . . . . . . . + 
10 + . . + . . . . . . + . . 
 9 . . . . . . . + . . . . . 
 8 . . + . . . . . . . . . . 
 7 . . . . . . . . . . . . . 
 6 . . . . . . . . . . . . . 
 5 . . + . . . . . . . . . . 
 4 . . . . . . . + . . . . . 
 3 + . . + . . . . . . + . . 
 2 . . . . . . . . . . . . + 
 1 . + . . . . . . . . . . . 
 0 . . . . . . . . . . . . . 
   0 1 2 3 4 5 6 7 8 9 101112

In [76]:
from tinyec.ec import SubGroup, Curve

a = 7
b = 3
p = 13

points = list_points(a, b, p)
print(points)


field = SubGroup(p=13, g=(0, 9), n=13, h=1)
curve = Curve(a=7, b=3, field=field, name='p1373')

# for curr_point in points:
#     field = SubGroup(p=p, g=curr_point, n=len(points)+1, h=1)
#     curve = Curve(a=a, b=b, field=field, name='ppp')
#     counter = 1
#     for k in range(1, 20):
#         p = k * curve.g
#         if p.x != None and p.y != None:
#             counter += 1
#         else:
#             break
#     print("{} | {}".format(curr_point, counter))


counter = 1
for k in range(1, 25):
    p = k * curve.g
    print(f"{k} * G = ({p.x}, {p.y})")
    if p.x != None and p.y != None:
        counter += 1
    else:
        break
print(counter)

[(0, 4), (0, 9), (2, 5), (2, 8), (3, 5), (3, 8), (4, 2), (4, 11), (6, 1), (6, 12), (8, 5), (8, 8)]
1 * G = (0, 9)
2 * G = (3, 5)
3 * G = (6, 12)
4 * G = (4, 2)
5 * G = (8, 5)
6 * G = (2, 5)
7 * G = (2, 8)
8 * G = (8, 8)
9 * G = (4, 11)
10 * G = (6, 1)
11 * G = (3, 8)
12 * G = (0, 4)
13 * G = (None, None)
13


## Adding points on a curve

### Points addition

> Let define two points on the elliptic curve *E*(*a*,*b*).
> These points are two distinct points $P(x_1,y_1)$ and $Q(x_2,y_2)$.\
> The calculation of the sum of these two points is given as follows.

$$ R(x_3,y_3)=P(x_1,y_1)+Q(x_2,y_2) $$

* if $x_1 \ne x_2$, the sum in this case is defined by

$$ x_3 = \lambda^2-x_1-x_2 $$
$$ y_3 = \lambda(x_1-x_3)-y_1 $$

where

$$ \lambda = \frac{y_2-y_1}{x_2-x_1} $$

* if $x_1 = x_2$, this case gives the point at infinity ($\theta$)

### Points doubling

> Let define two points on the elliptic curve *E*(*a*,*b*).
> These points are $P(x_1,y_1)$ and $Q(x_2,y_2)$, which are equal to each other i.e. $P=Q$.\
> Point doubling can be defined as follows.

$$ P(x_1,y_1)=Q(x_2,y_2) $$

where

$$ R(x_3,y_3)=P(x_1,y_1)+P(x_1,y_1)=2P $$

The calculation of the point doubling is given as follows.

$$ x_3 = \lambda^2-2x_1 $$
$$ y_3 = \lambda(x_1-x_3)-y_1 $$

where:

$$ \lambda = \frac{3x_1^2+a}{2y_1} \mod p $$



In [75]:
def calculate_modular_inverse(base, modulus):
    # Calculate the modular inverse using Extended Euclidean Algorithm
    a = base
    b = modulus
    x, y = 1, 0
    while b != 0:
        q = a // b
        a, b = b, a % b
        x, y = y, x - q * y
    
    if a == 1:
        # Ensure the modular inverse exists
        return x % modulus
    else:
        # Modular inverse does not exist
        return None

def sum_pt(a, p, point_1, point_2 = None):
    lam = ((3*point_1[0]**2 + a) * calculate_modular_inverse(2*point_1[1], p)) % p
    print("LAMBDA:", lam)
    
    x_3 = (lam**2 - 2*point_1[0]) % p
    y_3 = (lam*(point_1[0] - x_3) - point_1[1]) % p
    
    return (x_3, y_3)

init_point = (0, 9)

p_2 = sum_pt(a=7, p=13, point_1=init_point)

p_3 = sum_pt()
print(p_2)

"""
1 * G = (0, 9)
2 * G = (3, 5)
3 * G = (6, 12)
4 * G = (4, 2)
5 * G = (8, 5)
6 * G = (2, 5)
7 * G = (2, 8)
8 * G = (8, 8)
9 * G = (4, 11)
10 * G = (6, 1)
11 * G = (3, 8)
12 * G = (0, 4)
13 * G = (None, None)
"""

LAMBDA: 4
(3, 5)


In [74]:
def calculate_modular_inverse(base, modulus):
    # Calculate the modular inverse using Extended Euclidean Algorithm
    a = base
    b = modulus
    x, y = 1, 0
    while b != 0:
        q = a // b
        a, b = b, a % b
        x, y = y, x - q * y
    
    if a == 1:
        # Ensure the modular inverse exists
        return x % modulus
    else:
        # Modular inverse does not exist
        return None

base = 18
modulus = 13

modular_inverse = calculate_modular_inverse(base, modulus)
if modular_inverse is not None:
    print(f"The modular inverse of {base} modulo {modulus} is: {modular_inverse}")
else:
    print(f"The modular inverse of {base} modulo {modulus} does not exist.")


The modular inverse of 18 modulo 13 is: 8
